In [1]:
%run 'processing'
%run 'normalisasi'
%run 'case_folding'
%run 'stop_word'
%run 'stemmer'
%run 'bag_of_word'
%run 'word2vec'
%run 'transform_fitur'
from MyXML import MyXML

In [2]:
# load semua file and gabungin
data_opini_1 = MyXML('./data/anotated/2016.xml')
data_opini_2 = MyXML('./data/anotated/2017.xml')
data_opini_3 = MyXML('./data/anotated/2018.xml')
data_opini_4 = MyXML('./data/anotated/2019.xml')
data_opini = data_opini_1.get_pair().copy()
data_opini += data_opini_2.get_pair()
data_opini += data_opini_3.get_pair()
data_opini += data_opini_4.get_pair()

In [3]:
# split kalimat
data_opini = split_multi_aspek(data_opini)

In [4]:
# normalisasi
data_opini = normalisasi(data_opini)

In [6]:
# normalisasi pakai inaNLP
data_opini = ina_nlp_formalizer(data_opini)

In [7]:
# case folding
data_opini = case_folding_and_remove_number(data_opini)

lebih baik mahasiswa ditekankan untuk memakai  adab sebelum ilmu  atau etika dalam menuntut ilmu supaya kelas kondusif dan mendapatkan berkah  


In [9]:
# save data opini
save_data_opini(data_opini, 'data_opini_v2(baru).txt')

data_opini_v2(baru).txt saved


In [55]:
# load data opini
data_opini = open_data_opini('data_opini_v2(baru).txt')

In [56]:
# tf-idf
dict_tf_idf = tf_idf_dict(data_opini)
fitur_tf_idf = transform_bag_of_words(dict_tf_idf, data_opini)

In [57]:
#vectorizer = TfidfVectorizer()
#list_kalimat = []
#for text, opini in data_opini:
#    list_kalimat.append(text)
#x = vectorizer.fit_transform(list_kalimat)

In [58]:
#print(x.shape)
#print(vectorizer.get_feature_names())
#print(len(vectorizer.get_feature_names()))
#dict_tf_idf
#vectorizer.vocabulary_
#min(vectorizer.idf_)

In [59]:
# fitur postagger
fitur_pos_tag = transform_pos_tagger(data_opini)

In [60]:
# fitur word2vec
fitur_word2vec = transform_word2vec_average(data_opini)

In [61]:
# scaling tfidf
from sklearn import preprocessing
fitur_tf_idf = np.array(fitur_tf_idf)
fitur_tf_idf = preprocessing.scale(fitur_tf_idf)

In [62]:
# scaling postagger
fitur_pos_tag = np.array(fitur_pos_tag)
fitur_pos_tag = preprocessing.scale(fitur_pos_tag)

In [63]:
#fitur_pos_tag.mean(axis=0)

array([ 4.31485638e-15, -1.33473137e-15, -3.57088376e-16, -1.87382857e-15])

In [32]:
# scaling word2vec
fitur_word2vec = np.array(fitur_word2vec)
fitur_word2vec = preprocessing.scale(fitur_word2vec)

In [26]:
# gabung semua fitur
import numpy as np
X = []
for i in range(0, len(fitur_tf_idf)):
    X.append(np.concatenate((fitur_tf_idf[i], fitur_pos_tag[i], fitur_word2vec[i])))

In [27]:
# transform label
Y = transform_label(data_opini)
le = preprocessing.LabelEncoder()
le.fit(Y)
YY = le.transform(Y)

In [28]:
# split data test
from sklearn.model_selection import train_test_split
X_train , X_test , y_train, y_test = train_test_split(X, YY, random_state=109, test_size=0.3)

In [29]:
# training
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo', kernel='rbf')
clf.fit(X_train, y_train)

SVC(decision_function_shape='ovo')

In [30]:
# test
y_pred = clf.predict(X_test)

In [31]:
#show acuraccy
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average='micro'))
print("Recall:",metrics.recall_score(y_test, y_pred, average='micro'))

Accuracy: 0.674364896073903
Precision: 0.674364896073903
Recall: 0.674364896073903


In [23]:
# play with model
kalimat = "ruangan sangat panas"
data = [[kalimat, [['NoAspect', 'Positif']]]]
kalimat_tfidf = transform_bag_of_words(dict_tf_idf, data)
for i in range(0, 96 - len(kalimat_tfidf[0])):
    kalimat_tfidf[0].append(0.0)
kalimat_pos_tag = transform_pos_tagger(data)
kalimat_word2vec = transform_word2vec_average(data)

kalimat_tfidf = np.array(kalimat_tfidf)
kalimat_tfidf = preprocessing.scale(kalimat_tfidf)

kalimat_pos_tag = np.array(kalimat_pos_tag)
kalimat_pos_tag = preprocessing.scale(kalimat_pos_tag)

kalimat_word2vec = np.array(kalimat_word2vec)
kalimat_word2vec = preprocessing.scale(kalimat_word2vec)

In [24]:
import numpy as np
A = []
for i in range(0, len(kalimat_tfidf)):
    A.append(np.concatenate((kalimat_tfidf[i], kalimat_pos_tag[i], kalimat_word2vec[i])))